# Taller 4b
Importamos librerías a utilizar:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from IPython.display import display

Abrimos la base de datos y exploramos su contenido:

In [2]:
df = pd.read_csv("data.csv")
print(f"N° de datos: {len(df)}")
display(df.head())
display(df.describe())

N° de datos: 2947


,Year,Month,Day,O3,PM2.5,Environmental_risk
0,2008,1,1,29.63,NaN,NaN
1,2008,1,2,21.46,NaN,NaN
2,2008,1,3,24.25,NaN,NaN
3,2008,1,4,29.04,NaN,NaN
4,2008,1,5,30.17,NaN,NaN


,Year,Month,Day,O3,PM2.5
count,2947.000000,2947.000000,2947.000000,2878.000000,2725.000000
mean,2011.576179,6.405158,15.721751,16.313767,28.814594
std,2.350026,3.482292,8.782308,7.995680,17.479388
min,2008.000000,1.000000,1.000000,1.000000,5.000000
25%,2010.000000,3.000000,8.000000,9.392500,16.170000
50%,2012.000000,6.000000,16.000000,16.960000,23.210000
75%,2014.000000,9.000000,23.000000,22.357500,37.210000
max,2016.000000,12.000000,31.000000,56.300000,126.200000


Normalizamos los valores de las columnas categóricas. Para ``Enviromental_risk`` hay que tener un cuidado y es que posee distintos tipos de datos (``str`` para las categorías y ``float`` para los ``NaN``). De esta manera, pasamos todo a ``str`` y luego normalizamos, teniendo cuidado de identificar qué etiqueta se le asignó a los valores ``NaN``:

In [3]:
label_encoder = LabelEncoder()
for i in ["Year", "Month", "Day"]:
    df[i] = label_encoder.fit_transform(df[i])
df["Environmental_risk"] = label_encoder.fit_transform(df["Environmental_risk"].astype(str))
df.head()

,Year,Month,Day,O3,PM2.5,Environmental_risk
0,0,0,0,29.63,NaN,4
1,0,0,1,21.46,NaN,4
2,0,0,2,24.25,NaN,4
3,0,0,3,29.04,NaN,4
4,0,0,4,30.17,NaN,4


De lo anterior tenemos que se le asignó una etiqueta de 4 a los valores ``NaN``.

## Misión 1

Consideramos los registros que no tienen valores faltantes para ``Year``, ``Month``, ``Day``, ``O3`` y ``PM2.5`` y creamos un set de entrenamiento y otro de testeo:

In [4]:
cond1 = ~(df["Year"].isna() | df["Month"].isna() | df["Day"].isna() | df["O3"].isna() | df["PM2.5"].isna())
df_pred = df[cond1].copy()
training_set, test_set = train_test_split(df_pred.copy(), test_size = 0.3)

print(f'Tamaño set entrenamiento: {len(training_set)}')
print(f'Tamaño set test: {len(test_set)}')

Tamaño set entrenamiento: 1859
Tamaño set test: 797


Normalizamos la variable numérica que se utilizará como feature (``O3``):

In [5]:
scaler = StandardScaler()
var_num = ["O3"]

training_set[var_num] = scaler.fit_transform(training_set[var_num])
test_set[var_num] = scaler.transform(test_set[var_num])

training_set.head()

,Year,Month,Day,O3,PM2.5,Environmental_risk
494,1,4,8,-0.400913,32.79,3
1787,4,11,8,1.916133,16.62,2
2132,5,10,18,0.862251,21.79,3
1574,4,4,9,-1.104746,34.31,3
1810,5,0,0,1.532450,27.54,0


Implementamos una regresión de mínimos cuadrados ordinarios (https://scikit-learn.org/stable/modules/linear_model.html#ordinary-least-squares):

In [6]:
features = ["Year", "Month", "Day", "O3"]
target = "PM2.5"
reg1 = LinearRegression()
reg1.fit(training_set[features], training_set[target])
predictions = reg1.predict(test_set[features])
mse = metrics.mean_squared_error(test_set[target], predictions)
print(f"Error cuadrático medio: {mse}")

Error cuadrático medio: 234.04407114991616


Implementamos un modelo de regresión basado en redes neuronales (https://scikit-learn.org/stable/modules/neural_networks_supervised.html#regression):

In [7]:
reg2 = MLPRegressor(random_state=1, max_iter=500)
reg2.fit(training_set[features], training_set[target])
predictions = reg2.predict(test_set[features])
mse = metrics.mean_squared_error(test_set[target], predictions)
print(f"Error cuadrático medio: {mse}")

Error cuadrático medio: 175.22068496840828


c:\users\matia\appdata\local\programs\python\python36\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


El que tiene menor error cuadrático medio es el segundo modelo. Lo utilizamos para completar datos faltantes:

In [8]:
df["O3_norm"] = scaler.transform(df[var_num])
cond2 = ~(df["Year"].isna() | df["Month"].isna() | df["Day"].isna() | df["O3"].isna()) & df["PM2.5"].isna()
df.loc[cond2, "PM2.5"] = reg2.predict(df[cond2][["Year", "Month", "Day", "O3_norm"]])

## Misión 2

Hacemos exactamente lo mismo, solo que ahora para predecir valores de ``O3``:

In [9]:
cond1 = ~(df["Year"].isna() | df["Month"].isna() | df["Day"].isna() | df["O3"].isna() | df["PM2.5"].isna())
df_pred = df[cond1].copy()
training_set, test_set = train_test_split(df_pred.copy(), test_size = 0.3)

print(f'Tamaño set entrenamiento: {len(training_set)}')
print(f'Tamaño set test: {len(test_set)}')

Tamaño set entrenamiento: 2014
Tamaño set test: 864


In [10]:
scaler = StandardScaler()
var_num = ["PM2.5"]

training_set[var_num] = scaler.fit_transform(training_set[var_num])
test_set[var_num] = scaler.transform(test_set[var_num])

training_set.head()

,Year,Month,Day,O3,PM2.5,Environmental_risk,O3_norm
2527,6,11,18,15.75,-0.891352,4,-0.048373
1715,4,8,27,19.04,-0.853959,1,0.361470
2520,6,11,11,20.17,-0.773042,1,0.502237
336,0,11,1,24.17,-0.562080,1,1.000526
2497,6,10,18,20.80,-0.746294,4,0.580718


In [11]:
features = ["Year", "Month", "Day", "PM2.5"]
target = "O3"
reg1 = LinearRegression()
reg1.fit(training_set[features], training_set[target])
predictions = reg1.predict(test_set[features])
mse = metrics.mean_squared_error(test_set[target], predictions)
print(f"Error cuadrático medio: {mse}")

Error cuadrático medio: 42.375098952826065


In [12]:
reg2 = MLPRegressor(random_state=1, max_iter=500)
reg2.fit(training_set[features], training_set[target])
predictions = reg2.predict(test_set[features])
mse = metrics.mean_squared_error(test_set[target], predictions)
print(f"Error cuadrático medio: {mse}")

Error cuadrático medio: 19.360475816069428


El que tiene menor error cuadrático medio es el segundo modelo. Lo utilizamos para completar datos faltantes:

In [13]:
df["PM2.5_norm"] = scaler.transform(df[var_num])
cond2 = ~(df["Year"].isna() | df["Month"].isna() | df["Day"].isna() | df["PM2.5"].isna()) & df["O3"].isna()
df.loc[cond2, "O3"] = reg2.predict(df[cond2][["Year", "Month", "Day", "PM2.5_norm"]])

## Misión 3

Consideramos todas las observaciones con etiqueta distinta de 4 en ``Environmental_risk`` (recordar que son los NaN) y creamos un set de entrenamiento y testeo:

In [14]:
cond1 = df["Environmental_risk"] != 4
df_pred = df[cond1].copy()
training_set, test_set = train_test_split(df_pred.copy(), test_size = 0.3)

print(f'Tamaño set entrenamiento: {len(training_set)}')
print(f'Tamaño set test: {len(test_set)}')

Tamaño set entrenamiento: 1765
Tamaño set test: 757


Normalizamos:

In [15]:
scaler = StandardScaler()

var_num = ["O3", "PM2.5"]
training_set[var_num] = scaler.fit_transform(training_set[var_num])
test_set[var_num] = scaler.transform(test_set[var_num])

training_set.head()

,Year,Month,Day,O3,PM2.5,Environmental_risk,O3_norm,PM2.5_norm
1690,4,8,2,-0.711089,-0.661616,1,-0.779613,-0.610630
2291,6,3,26,-0.150196,-0.397958,3,-0.207825,-0.337246
133,0,4,12,-0.339604,1.633270,3,-0.400913,1.768907
975,2,8,14,-0.064657,-0.026160,3,-0.120625,0.048266
1984,5,5,23,-1.312307,-0.051802,3,-1.392509,0.021679


Implementamos un árbol de decisión como primer modelo (https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier):

In [16]:
features = ["Year", "Month", "Day", "O3", "PM2.5"]
target = "Environmental_risk"
model1 = DecisionTreeClassifier()
model1.fit(training_set[features], training_set[target])
predictions = model1.predict(test_set[features])
bas = metrics.balanced_accuracy_score(test_set[target], predictions)
print(f"Balanced accuracy: {bas}")

Balanced accuracy: 0.9655405405405406


Ahora implementamos un modelo de K vecinos más cercanos (https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html):

In [17]:
model2 = KNeighborsClassifier()
model2.fit(training_set[features], training_set[target])
predictions = model2.predict(test_set[features])
bas = metrics.balanced_accuracy_score(test_set[target], predictions)
print(f"Balanced accuracy: {bas}")

Balanced accuracy: 0.6034391829846375


Usamos el modelo 1 que tiene mejor rendimiento para completar los datos faltantes:

In [18]:
df[["O3_norm", "PM2.5_norm"]] = scaler.transform(df[var_num])
cond = df["Environmental_risk"] == 4
df.loc[cond, "Environmental_risk"] = model1.predict(df[cond][["Year", "Month", "Day", "O3_norm", "PM2.5_norm"]])

## Misión 4

Veamos las nuevas frecuencias de las categorías:

In [19]:
df["Environmental_risk"].value_counts()

3    1781
1     830
0     274
2      62
Name: Environmental_risk, dtype: int64

Observamos que el modelo predictivo asignó de todas las categorías a los datos faltantes, mientras que en la T4a habíamos logrado desagregar los niveles de bajo, extremo y medio, sin asignar nunca un nivel de riesgo alto. Además, a pesar que definimos una condición para el nivel extremo, no existían datos faltantes que cumplieran esa condición, por lo que no se asignó nunca este nivel a datos faltantes.

El esquema de asignación planteado en la T4a era el siguiente:

- Si la cantidad de ``O3`` es mayor que 1.38095622 y la cantidad de ``PM2.5`` es menor que -0.65302355, asignar nivel ``extremo``.
- Si la cantidad de ``O3`` es menor que 1.38095622 y la cantidad de ``PM2.5`` es menor que -0.65302355, asignar nivel ``Bajo``.
- Asignar nivel ``Medio`` en otro caso.

Notar que se estandarizó los valores para trabajar con la estandarización aplicada al inicio en el DataFrame. Para comparar este esquema con el modelo escogido, aplicamos el modelo sobre el set de testeo y evaluamos su rendimiento:

In [22]:
scaler.transform(np.array([[27,np.exp(2.9)]]))

array([[ 1.38095622, -0.65302355]])

In [23]:
test_esquema = test_set.copy()
cond1 = (test_esquema["O3"] < 1.38095622) & (test_esquema["PM2.5"] < -0.65302355)
cond2 = (test_esquema["O3"] > 1.38095622) & (test_esquema["PM2.5"] < -0.65302355)
cond3 = ~(cond1 | cond2)
test_esquema.loc[cond1, "Environmental_risk"] = 1
test_esquema.loc[cond2, "Environmental_risk"] = 2
test_esquema.loc[cond3, "Environmental_risk"] = 3
metrics.balanced_accuracy_score(test_set["Environmental_risk"], test_esquema["Environmental_risk"])

0.6975225225225226

El rendimiento del modelo predictivo es mucho mejor que el del esquema planteado para el T4a.